# Modeling
ML Tasks

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline

## Input

In [2]:
from sklearn.datasets import load_files

corpus = load_files("../data/")

doc_count = len(corpus.data)
print("Doc count:", doc_count)
assert doc_count is 56, "Wrong number of documents loaded, should be 56 (56 stories)"

Doc count: 56


## Vectorizer

In [14]:
from tokenizer import TextWrangler
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

bow_pure = CountVectorizer(strip_accents="ascii", lowercase=True, stop_words=None)
X_bow_pure = bow_pure.fit_transform(corpus.data)

bow_stem = CountVectorizer(strip_accents="ascii", tokenizer=TextWrangler(kind="stem"))
X_bow_stem = bow_stem.fit_transform(corpus.data)

tfidf_stem = TfidfVectorizer(strip_accents="ascii", tokenizer=TextWrangler(kind="stem"))
X_tfidf_stem = tfidf_stem.fit_transform(corpus.data)

bow_lemma = CountVectorizer(strip_accents="ascii", tokenizer=TextWrangler(kind="lemma"))
X_bow_lemma = bow_lemma.fit_transform(corpus.data)

tfidf_lemma = TfidfVectorizer(strip_accents="ascii", tokenizer=TextWrangler(kind="lemma"))
X_tfidf_lemma = tfidf_lemma.fit_transform(corpus.data)

## Models

In [15]:
from sklearn.decomposition import LatentDirichletAllocation, TruncatedSVD, NMF

n_topics = 5

lda = LatentDirichletAllocation(n_components=n_topics)
lsa = TruncatedSVD()
nmf = NMF()

In [16]:
lda_params = {"lda__learning_decay": [0.5, 0.7, 0.9],
              "lda__learning_offset": [1., 5., 10.]}

## Pipeline

In [17]:
from sklearn.pipeline import Pipeline

lda_pipe = Pipeline([
    ("bow", bow_stem),
    ("lda", lda)
])

## Gridsearch

In [18]:
from sklearn.model_selection import GridSearchCV

lda_model = GridSearchCV(lda_pipe, param_grid=lda_params, cv=5, n_jobs=-1)

## Training

In [19]:
lda_model.fit(corpus.data)

/home/datadonk23/anaconda3/envs/holmes-topic-models/lib/python3.6/site-packages/sklearn/model_selection/_search.py:841: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=5, error_score='raise-deprecating',
       estimator=Pipeline(memory=None,
     steps=[('bow', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
        strip_...        random_state=None, topic_word_prior=None,
             total_samples=1000000.0, verbose=0))]),
       fit_params=None, iid='warn', n_jobs=-1,
       param_grid={'lda__learning_decay': [0.5, 0.7, 0.9], 'lda__learning_offset': [1.0, 5.0, 10.0]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

In [20]:
print(lda_model.best_params_)
lda_model.best_score_

{'lda__learning_decay': 0.9, 'lda__learning_offset': 1.0}


-268629.3343791538

## Evaluation

In [10]:
#print("Log Likelihood:", model.score(corpus.data))

In [21]:
def show_topics(vectorizer, lda_model, n_words=20):
    keywords = np.array(vectorizer.get_feature_names())
    topic_keywords = []
    for topic_weights in lda_model.components_:
        top_keyword_locs = (-topic_weights).argsort()[:n_words]
        topic_keywords.append(keywords.take(top_keyword_locs))
    return topic_keywords

topic_keywords = show_topics(vectorizer=bow_stem, lda_model=lda_model.best_estimator_.named_steps.lda, n_words=15)        

df_topic_keywords = pd.DataFrame(topic_keywords)
df_topic_keywords.columns = ['Word '+str(i) for i in range(df_topic_keywords.shape[1])]
df_topic_keywords.index = ['Topic '+str(i) for i in range(df_topic_keywords.shape[0])]
df_topic_keywords

,Word 0,Word 1,Word 2,Word 3,Word 4,Word 5,Word 6,Word 7,Word 8,Word 9,Word 10,Word 11,Word 12,Word 13,Word 14
Topic 0,hand,room,know,com,littl,look,let,think,door,fac,tim,cas,mat,cam,hous
Topic 1,com,room,hand,way,tim,hous,cas,fac,pap,look,think,know,night,let,littl
Topic 2,garrideb,trev,hudson,beddo,am,prescot,nath,convict,glor,scot,prendergast,seam,sail,ship,chaplain
Topic 3,rent,evolv,intuit,upright,scheming,discharg,belt,val,benefact,leaf,healthy,clasp,chimney,sideboard,streaming
Topic 4,com,hand,know,look,room,fac,tim,littl,day,good,sir,think,say,way,hous


In [22]:
import pyLDAvis
from pyLDAvis.sklearn import prepare
pyLDAvis.enable_notebook()

prepare(lda_model.best_estimator_.named_steps.lda, X_bow_stem, bow_stem, mds="tsne")

/home/datadonk23/anaconda3/envs/holmes-topic-models/lib/python3.6/site-packages/pyLDAvis/_prepare.py:257: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  return pd.concat([default_term_info] + list(topic_dfs))


PreparedData(topic_coordinates=                x           y  topics  cluster       Freq
topic                                                    
4      147.645020 -158.535141       1        1  44.160694
0      -13.247591    4.243062       2        1  33.841102
1       -8.174920 -224.607864       3        1  21.559695
2      156.025726   16.866489       4        1   0.431750
3     -140.024979 -108.624443       5        1   0.006759, topic_info=     Category        Freq       Term       Total  loglift  logprob
term                                                              
201   Default   72.000000         am   72.000000  30.0000  30.0000
3518  Default   43.000000     hudson   43.000000  29.0000  29.0000
4521  Default   75.000000       mess   75.000000  28.0000  28.0000
2615  Default  161.000000       fath  161.000000  27.0000  27.0000
6377  Default   40.000000       ship   40.000000  26.0000  26.0000
5560  Default   63.000000     prison   63.000000  25.0000  25.0000
4067  Default   39.000000       land   39.000000  24.0000  24.0000
4109  Default   69.000000        law   69.000000  23.0000  23.0000
6401  Default   76.000000       shot   76.000000  22.0000  22.0000
6109  Default   33.000000       sail   33.000000  21.0000  21.0000
5367  Default   31.000000     pistol   31.000000  20.0000  20.0000
1386  Default   48.000000    collect   48.000000  19.0000  19.0000
4028  Default   59.000000      knock   59.000000  18.0000  18.0000
2099  Default   56.000000       docu   56.000000  17.0000  17.0000
2972  Default   64.000000        gam   64.000000  16.0000  16.0000
758   Default   28.000000       boat   28.000000  15.0000  15.0000
7472  Default   23.000000       trev   23.000000  14.0000  14.0000
4633  Default  133.000000      money  133.000000  13.0000  13.0000
6201  Default   27.000000       scot   27.000000  12.0000  12.0000
3999  Default   60.000000        kil   60.000000  11.0000  11.0000
6386  Default   34.000000      shock   34.000000  10.0000  10.0000
4166  Default  180.000000    lestrad  180.000000   9.0000   9.0000
5139  Default  446.000000        pap  446.000000   8.0000   8.0000
6637  Default  130.000000        son  130.000000   7.0000   7.0000
5760  Default  240.000000       read  240.000000   6.0000   6.0000
3092  Default   71.000000    godfrey   71.000000   5.0000   5.0000
4884  Default   32.000000        nor   32.000000   4.0000   4.0000
1538  Default   21.000000    convict   21.000000   3.0000   3.0000
3125  Default   25.000000     govern   25.000000   2.0000   2.0000
902   Default  106.000000      broth  106.000000   1.0000   1.0000
...       ...         ...        ...         ...      ...      ...
5887   Topic5    0.001355       rent   14.213884   0.3440  -9.0199
2477   Topic5    0.001355      evolv    5.505378   1.2925  -9.0199
3826   Topic5    0.001355     intuit    7.417701   0.9944  -9.0199
7797   Topic5    0.001355    upright    3.514605   1.7413  -9.0199
6181   Topic5    0.001355   scheming    7.416550   0.9945  -9.0199
1994   Topic5    0.001355   discharg    7.384744   0.9988  -9.0199
611    Topic5    0.001355       belt    4.549613   1.4832  -9.0199
7821   Topic5    0.001355        val    7.460340   0.9887  -9.0199
617    Topic5    0.001355   benefact    7.460339   0.9887  -9.0199
4120   Topic5    0.001355       leaf    8.416059   0.8681  -9.0199
3358   Topic5    0.001355    healthy    7.460116   0.9887  -9.0199
1299   Topic5    0.001355      clasp   15.147261   0.2804  -9.0199
1228   Topic5    0.001355    chimney   10.342901   0.6620  -9.0199
6442   Topic5    0.001355  sideboard   11.343347   0.5696  -9.0199
6916   Topic5    0.001355  streaming    2.568912   2.0548  -9.0199
3482   Topic5    0.001355      horac    8.305893   0.8813  -9.0199
1499   Topic5    0.001355    conspir    8.668440   0.8386  -9.0199
7287   Topic5    0.001355    thready    2.569409   2.0546  -9.0199
5283   Topic5    0.001355     persev    3.543313   1.7332  -9.0199
3765   Topic5    0.001355     insert    2.569365   2.0546  -9.0199
